In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import plotly.graph_objs as go
import plotly.plotly as py

In [2]:
pd.set_option('display.max_rows', 2000)

# Non-Performing Loans

#### Read data

In [3]:
_HDFStore = pd.HDFStore("npl_loans_subset.h5")
df_np = _HDFStore['df']
_HDFStore.close()

#### View features

In [4]:
df_np.columns.values

array(['svcg_cycle', 'st', 'occr_default_per_state',
       'occr_default_per_state_12_mon', 'occr_paid_off_per_state',
       'occr_paid_off_per_state_12_mon', 'new_loans_per_state_12_mon',
       'active_loans_per_state', 'new_loans_per_state', 'zipcode',
       'occr_default_per_zipcode', 'occr_default_per_zipcode_12_mon',
       'occr_paid_off_per_zipcode', 'occr_paid_off_per_zipcode_12_mon',
       'new_loans_per_zipcode_12_mon', 'active_loans_per_zipcode',
       'new_loans_per_zipcode', 'nat_int_rt', 'unemploy_rt',
       'hous_prc_indx_st', 'label_good_bad_loan', 'id_loan',
       'label_month_final', 'fico', 'dt_first_pi', 'flag_fthb', 'dt_matr',
       'cd_msa', 'mi_pct', 'cnt_units', 'occpy_sts', 'cltv', 'dti',
       'orig_upb', 'ltv', 'int_rt', 'channel', 'ppmt_pnlty', 'prod_type',
       'prop_type', 'loan_purpose', 'orig_loan_term', 'cnt_borr',
       'flag_sc', 'current_upb', 'delq_sts', 'loan_age', 'mths_remng',
       'repch_flag', 'flag_mod', 'cd_zero_bal', 'dt_zero_

#### Print general information

In [5]:
LOAN_COUNT = len(np.unique(df_np['id_loan'].values))
FICO_MEAN = df_np['fico'].mean()
BALANCE_MEAN = df_np['orig_upb'].mean()
FICO_MEDIAN = df_np['fico'].median()
FICO_MEDIAN = df_np['fico'].median()
LOAN_LENGTH_MEAN = df_np.groupby(['id_loan']).size().mean()
DEFAULT_LOANS_COUNT = len(df_np.loc[df_np['label_good_bad_loan'] == 0]['id_loan'].unique())
FULLY_PAID_LOANS_COUNT = len(df_np.loc[df_np['label_good_bad_loan'] == 1]['id_loan'].unique())

print "FICO_MEDIAN: " + str(int(FICO_MEDIAN))
print "FICO_MEAN: " + str(int(FICO_MEAN))
print "LOAN_COUNT: " + str(int(LOAN_COUNT))
print "ORIGINAL_BALANCE_MEAN: $" + str(int(BALANCE_MEAN))
print "LOAN_LENGTH_MEAN (months): " + str(int(LOAN_LENGTH_MEAN))
print "Number of Default Loans :" + str(int(DEFAULT_LOANS_COUNT))
print "Number of Fully Paid Loans :" + str(int(FULLY_PAID_LOANS_COUNT))


FICO_MEDIAN: 670
FICO_MEAN: 670
LOAN_COUNT: 398
ORIGINAL_BALANCE_MEAN: $137098
LOAN_LENGTH_MEAN (months): 21
Number of Default Loans :83
Number of Fully Paid Loans :315


#### Prepare Data for graphs

In [6]:
def st_names():
    return pd.read_table("st_names.csv", sep=',', names=['st', 'st_name'])

def fill_dataframe_to_st(df_IN):
    df_st_names = st_names()
    df_filled = pd.merge(df_st_names, df_IN, on='st', how='outer')
    for col in df_filled.columns.values:
        df_filled[col] = df_filled[col].fillna(0)
    return df_filled

#  Calculate number of npls per state 
df_st_count = df_np.drop_duplicates(subset=['id_loan']).groupby(['st']).size().to_frame()
df_st_count.reset_index(level=0, inplace=True)
df_st_count.columns = ['st', 'st_loan_count']
df_np = pd.merge(df_np, df_st_count, on='st', how='left')
df_np['st_loan_count']= df_np['st_loan_count'].fillna(0)

df_st = df_np.sort_values(['st_loan_count'], ascending=[False])
df_st = df_st.drop_duplicates(subset=['st'], keep='first')
df_st = df_st.loc[df_st['st'] != 'PR']
df_st = fill_dataframe_to_st(df_st)
df_st = df_st.sort_values(['st_loan_count'], ascending=[True])

st_col = df_st['st'].values
occr_loans_per_state_col = df_st['st_loan_count'].values


In [7]:
df = df_st[['st', 'st_name', 'st_loan_count', 'rt_default_per_state']]
df = df.rename(columns={'st': 'code'})

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['st_name'] + '<br>' + 'Default Rate : ' + df['rt_default_per_state'].astype(float).round(4).astype(str)

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['code'],
        z = df['st_loan_count'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "#")
        ) ]

layout = dict(
        title = 'Number of Loans by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [8]:
trace1 = go.Bar(
    x=df_st['st_loan_count'].values,
    y=df_st['st'].values,
    name='st_loan_count',
    orientation = 'h',
)
data = [trace1]
layout = go.Layout(
    autosize=False,
    width=500,
    height=1000,
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='horizontal-stacked-bar')